In [90]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
import joblib

In [91]:
# Load dataset
df = pd.read_csv("../Datasets/vitals.csv")

# Features
X = df[[
    "heart_rate",
    "resp_rate",
    "blood_pressure_systolic",
    "blood_pressure_diastolic",
    "spo2",
    "temperature_c",
    "glucose_mgdl"
]]

In [92]:

# Encode labels
le_status = LabelEncoder()
le_condition = LabelEncoder()

y_status = le_status.fit_transform(df["label_status"])            
y_condition = le_condition.fit_transform(df["probable_condition"])  

# Combine into one DataFrame (multi-output target)
y = pd.DataFrame({
    "label_status": y_status,
    "probable_condition": y_condition
})

In [93]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [94]:
# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [95]:
# Train model
clf = MultiOutputClassifier(RandomForestClassifier(random_state=42))
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Reports
print("Classification Report for Label Status (Normal/Warning/Critical):")
print(classification_report(y_test["label_status"], y_pred[:,0], target_names=le_status.classes_))

print("\nClassification Report for Probable Condition (Healthy/Not Healthy):")
print(classification_report(y_test["probable_condition"], y_pred[:,1], target_names=le_condition.classes_))

Classification Report for Label Status (Normal/Warning/Critical):
              precision    recall  f1-score   support

    Critical       0.97      1.00      0.98        30
      Normal       0.98      1.00      0.99        56
     Warning       1.00      0.86      0.92        14

    accuracy                           0.98       100
   macro avg       0.98      0.95      0.97       100
weighted avg       0.98      0.98      0.98       100


Classification Report for Probable Condition (Healthy/Not Healthy):
              precision    recall  f1-score   support

     Healthy       1.00      1.00      1.00        56
 Not Healthy       1.00      1.00      1.00        44

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [96]:
# --- Save model, scaler, and encoders ---
joblib.dump(clf, "../Models/multioutput_model.pkl")
joblib.dump(scaler, "../Models/scaler.pkl")
joblib.dump(le_status, "../Models/label_encoder_status.pkl")
joblib.dump(le_condition, "../Models/label_encoder_condition.pkl")
print("\n✅ Model and preprocessing objects saved successfully!")


✅ Model and preprocessing objects saved successfully!


In [97]:
accuracy_status = (y_test["label_status"] == y_pred[:,0]).mean()
accuracy_condition = (y_test["probable_condition"] == y_pred[:,1]).mean()

print(f"Accuracy (Label Status): {accuracy_status:.2f}")
print(f"Accuracy (Probable Condition): {accuracy_condition:.2f}")

Accuracy (Label Status): 0.98
Accuracy (Probable Condition): 1.00
